In [0]:
import pandas as pd
import requests
import time
from datetime import datetime
from pyspark.sql.functions import *

In [0]:
df_weather_spark = spark.sql(f"""SELECT * FROM delta. `/dbfs/Bronze/ExtraçãoTempo` """)

In [0]:
df_weather_spark.limit(5).display()

city,latitude,longitude,datetime,temperature,evapotranspiration,precipitation,humidity
São Paulo,-10.54156,-37.54603,2025-01-14T08:00,26.6,0.29,0.4,74
Rio de Janeiro,-22.89550,-43.40681,2025-01-27T12:00,30.7,0.53,0.0,62
Rio de Janeiro,-22.89550,-43.40681,2025-01-17T13:00,34.4,0.84,0.0,43
Rio de Janeiro,-22.89550,-43.40681,2025-01-17T08:00,25.3,0.22,0.0,86
Rio de Janeiro,-22.89550,-43.40681,2025-01-15T15:00,26.4,0.24,0.0,66


In [0]:
df_weather_spark = df_weather_spark.withColumn("date_str", split(col("datetime"), "T")[0]) \
       .withColumn("time_str", split(col("datetime"), "T")[1])

df_weather_spark = df_weather_spark.withColumn("date", to_date(col("date_str"), "yyyy-MM-dd"))

In [0]:
df_weather_spark = df_weather_spark.drop('datetime','date_str','latitude','longitude')

In [0]:
df_weather_spark.limit(5).display()

city,temperature,evapotranspiration,precipitation,humidity,time_str,date
São Paulo,26.6,0.29,0.4,74,08:00,2025-01-14
Rio de Janeiro,30.7,0.53,0.0,62,12:00,2025-01-27
Rio de Janeiro,34.4,0.84,0.0,43,13:00,2025-01-17
Rio de Janeiro,25.3,0.22,0.0,86,08:00,2025-01-17
Rio de Janeiro,26.4,0.24,0.0,66,15:00,2025-01-15


In [0]:
df_weather_spark.repartition(20)\
              .write.format("delta")\
              .mode("overwrite")\
              .option("overwriteSchema", "true")\
              .save("/dbfs/Silver/ExtraçãoTempo")